## Decision Trees

Decision trees are basically just nested if statements. The conditions in this if statements are what are learned by techniques that are rooted in Information Theory. 


### Information Theory
We measure weight by kilos, and water by litres, but how to measure information? The answer is Information Entropy. For instance, a page of a random book has more Entropy than the book itself because there's a lot of randomness in the page. On the other hand all the information is present in the book so it's obvious (less random = less Entropy)  


Entropy is used to choose the best splits in the DT. Our motive is to choose a split that maximizes the reduction in uncertainity. Eg: going from 50% certain to 100% certain is better than going from 50% to 75%.

